# Navigate to the directory of this file

It is expected that you will have downloaded the zip and uploaded to your own google drive. Then you will have to mount to your drive.

In [1]:
#If necessary, you can run this command to mount your drive.

from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [2]:
#Navigate to the directory of this file.

%cd /content/gdrive/MyDrive/ECE570/Project/EfficientNet/efficientnet
!ls

/content/gdrive/MyDrive/ECE570/Project/EfficientNet/efficientnet
best_models	     Nets		  run_experiment.py
find_hyperparams.py  print_parameters.py  RunExperiments.ipynb
joe_trained_models   ray_results.txt	  run_results_40_samples.ipynb
logs		     README.md		  trained_models
main.py		     RunExercises.ipynb   trainingjoe.ipynb


# Test a network's accuracy on CIFAR10

Import necessary modules and then make the /Nets directory useable in this file.

Run this first cell. Then optionally change the model_to_test to which model you would like to test. The available options to test are given in the dictionary below that line. 

Then run the cell and the test will run to find the network's accuracy on CIFAR 10.

In [4]:
import sys
sys.path.append('Nets/')

import Nets.EfficientNet as ef
import Nets.JoeNet as jn
import math
import torch
from torch import nn
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn.functional as F
import torchvision


In [ ]:
# Select one of the models from the dictionary below to test



#################### GRADER SHOULD CHANGE THIS ###############

model_to_test = "joenet-b2"

#################### END OF GRADER CHANGES ###################




models = {
  'joenet-b0': {
      "dims":(1.0, 1.0, 224),
      "directory":"joe_trained_models/JoeNet_b0_final_model.pt",
      "class":jn.JoeNet
  },
  'joenet-b1': {
      "dims":(1.1, 1.3, 224),
      "directory":"joe_trained_models/JoeNet_b1_final_model.pt",
      "class":jn.JoeNet
  },
  'joenet-b2': {
      "dims":(1.1, 1.6, 227),
      "directory":"joe_trained_models/JoeNet_b2_final_model.pt",
      "class":jn.JoeNet
  },
  'joenet-b3': {
      "dims":(1.2, 2.5, 233),
      "directory":"joe_trained_models/JoeNet_b3_final_model.pt",
      "class":jn.JoeNet
  },
  'joenet-b4': {
      "dims":(1.3, 4.0, 237),
      "directory":"joe_trained_models/JoeNet_b4_final_model.pt",
      "class":jn.JoeNet
  },
  'joenet-depth-scale' : {
      "dims":(1.0, 1.6, 224),
      "directory":"joe_trained_models/JoeNet_depth_scale.pt",
      "class":jn.JoeNet
  },
  'joenet-res-scale' : {
      "dims":(1.0, 1.0, 227),
      "directory":"joe_trained_models/JoeNet_res_scale.pt",
      "class":jn.JoeNet
  },
  'joenet-width-scale' : {
      "dims":(1.1, 1.0, 224),
      "directory":"joe_trained_models/JoeNet_width_scale.pt",
      "class":jn.JoeNet
  },
  'efficientnetb0': {
      "dims":(1.0, 1.0, 224),
      "directory":"trained_models/EfficientNet_final_model.pt",
      "class":ef.EfficientNet
  }
}

scale_width=models[model_to_test]['dims'][0]
scale_depth=models[model_to_test]['dims'][1]
resolution=models[model_to_test]['dims'][2]
model_directory = models[model_to_test]['directory']
model_class = models[model_to_test]["class"]

device = torch.device('cuda')

from torchvision.transforms.functional import InterpolationMode
transform = torchvision.transforms.Compose([
    torchvision.transforms.AutoAugment(policy=transforms.AutoAugmentPolicy.CIFAR10, 
                                       interpolation=InterpolationMode.BILINEAR),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Resize(resolution)])

test_dataset = torchvision.datasets.CIFAR10('/data', train=False, download=True, transform=transform)

batch_size = 100

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

def test_cifar(classifier, epoch):

  classifier.eval() 

  test_loss = 0
  correct = 0

  with torch.no_grad():
    for images, targets in test_loader:
      images = images.to(device)
      targets = targets.to(device)
      output = classifier(images)
      test_loss += F.cross_entropy(output, targets, reduction='sum').item()
      pred = output.data.max(1, keepdim=True)[1] 
      correct += pred.eq(targets.data.view_as(pred)).sum() 
  
  test_loss /= len(test_loader.dataset)

  print(f'Test result on epoch {epoch}: Avg loss is {test_loss}, Accuracy: {100.*correct/len(test_loader.dataset)}%')
  return correct, len(test_loader.dataset)

max_test_epoch = 1
num_correct, num_samples = 0,0
m = model_class(scale_width=scale_width, scale_depth=scale_depth, output_size=10)
path = model_directory
m.load_state_dict(torch.load(path))
print("Successfully loaded model")
m.to(device)
print("Beginning Test")
for epoch in range(1, max_test_epoch+1):
  correct, samples = test_cifar(m, epoch)
  num_correct+=correct
  num_samples+=samples

  print(f'Running accuracy on epoch {epoch}: {100.*num_correct/num_samples}%')

print(f'Overall accuracy is: {100.*num_correct/num_samples}%')

# Finding the number of Parameters

Testing the finding of number of parameters

In [26]:
!pip install torchinfo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [27]:
import sys
sys.path.append('Nets/')

import Nets.EfficientNet as ef
import Nets.JoeNet as jn
import math
import torch
from torch import nn
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn.functional as F
import torchvision
from torchinfo import summary

In [30]:
# Select one of the models from the dictionary below to test



#################### GRADER SHOULD CHANGE THIS ###############

model_to_test = "efficientnetb0"

#################### END OF GRADER CHANGES ###################




models = {
  'joenet-b0': {
      "dims":(1.0, 1.0, 224),
      "directory":"joe_trained_models/JoeNet_b0_final_model.pt",
      "class":jn.JoeNet
  },
  'joenet-b1': {
      "dims":(1.1, 1.3, 224),
      "directory":"joe_trained_models/JoeNet_b1_final_model.pt",
      "class":jn.JoeNet
  },
  'joenet-b2': {
      "dims":(1.1, 1.6, 227),
      "directory":"joe_trained_models/JoeNet_b2_final_model.pt",
      "class":jn.JoeNet
  },
  'joenet-b3': {
      "dims":(1.2, 2.5, 233),
      "directory":"joe_trained_models/JoeNet_b3_final_model.pt",
      "class":jn.JoeNet
  },
  'joenet-b4': {
      "dims":(1.3, 4.0, 237),
      "directory":"joe_trained_models/JoeNet_b4_final_model.pt",
      "class":jn.JoeNet
  },
  'joenet-depth-scale' : {
      "dims":(1.0, 1.6, 224),
      "directory":"joe_trained_models/JoeNet_depth_scale.pt",
      "class":jn.JoeNet
  },
  'joenet-res-scale' : {
      "dims":(1.0, 1.0, 227),
      "directory":"joe_trained_models/JoeNet_res_scale.pt",
      "class":jn.JoeNet
  },
  'joenet-width-scale' : {
      "dims":(1.1, 1.0, 224),
      "directory":"joe_trained_models/JoeNet_width_scale.pt",
      "class":jn.JoeNet
  },
  'efficientnetb0': {
      "dims":(1.0, 1.0, 224),
      "directory":"trained_models/EfficientNet_final_model.pt",
      "class":ef.EfficientNet
  }
}

scale_width=models[model_to_test]['dims'][0]
scale_depth=models[model_to_test]['dims'][1]
resolution=models[model_to_test]['dims'][2]
model_directory = models[model_to_test]['directory']
model_class = models[model_to_test]["class"]

device = torch.device('cuda')

m = model_class(scale_width=scale_width, scale_depth=scale_depth, output_size=10)
print(summary(m, input_size=(200,3,resolution,resolution)))

Layer (type:depth-idx)                                  Output Shape              Param #
EfficientNet                                            [200, 10]                 --
├─ConvBlock: 1-1                                        [200, 32, 112, 112]       --
│    └─Conv2d: 2-1                                      [200, 32, 112, 112]       864
│    └─BatchNorm2d: 2-2                                 [200, 32, 112, 112]       64
│    └─SiLU: 2-3                                        [200, 32, 112, 112]       --
├─Sequential: 1-2                                       [200, 1280, 7, 7]         --
│    └─Sequential: 2-4                                  [200, 16, 112, 112]       --
│    │    └─MBConvBlock: 3-1                            [200, 16, 112, 112]       1,448
│    └─Sequential: 2-5                                  [200, 24, 56, 56]         --
│    │    └─MBConvBlock: 3-2                            [200, 24, 56, 56]         6,004
│    │    └─MBConvBlock: 3-3                         

In [33]:
!git status

On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
  (use "git add/rm <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   Nets/__pycache__/BaseBlocks.cpython-37.pyc
	modified:   Nets/__pycache__/EfficientNet.cpython-37.pyc
	modified:   RunExercises.ipynb
	deleted:    runexercises.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	Nets/__pycache__/JoeNet.cpython-37.pyc
	best_models/
	logs/

no changes added to commit (use "git add" and/or "git commit -a")


In [32]:
!git add joe_trained_models/ RunExperiments.ipynb trainingjoe.ipynb RunExercises.ipynb
!git config --global user.email "pappas_joe@yahoo.com"
!git config --global user.name "Joe Pappas"
!git commit -m "commit experiment running"

!git push

[master f64a2c7] commit experiment running
 11 files changed, 3 insertions(+), 2 deletions(-)
 create mode 100644 RunExercises.ipynb
 rewrite RunExperiments.ipynb (98%)
 rename joe_trained_models/{JoeNet_b0_11_15_final_epoch_13.pt => JoeNet_b0_final_model.pt} (100%)
 rename joe_trained_models/{JoeNet_b1_11_15_final_epoch_13.pt => JoeNet_b1_final_model.pt} (100%)
 rename joe_trained_models/{JoeNet_b2_11_15_final_epoch_13.pt => JoeNet_b2_final_model.pt} (100%)
 rename joe_trained_models/{JoeNet_b3_11_14_final_epoch_13.pt => JoeNet_b3_final_model.pt} (100%)
 rename joe_trained_models/{JoeNet_b4_11_14_final_epoch_13.pt => JoeNet_b4_final_model.pt} (100%)
 rename joe_trained_models/{JoeNet_depth_scale_11_15_final_epoch_13.pt => JoeNet_depth_scale.pt} (100%)
 rename joe_trained_models/{JoeNet_res_scale_11_15_final_epoch_13.pt => JoeNet_res_scale.pt} (100%)
 rename joe_trained_models/{JoeNet_width_scale_11_15_final_epoch_13.pt => JoeNet_width_scale.pt} (100%)
 rewrite trainingjoe.ipynb (95%)
